In [ ]:
"""
    This notebook captures the process by which de-identification is ran against the AoUS bigquery database
    @TODO:
    - Need to add some form of logs to know what is going on at all times
    - 
"""
from __future__ import division
import json
from google.cloud import bigquery as bq

from deid import *
client = bq.Client.from_service_account_json('/home/steve/git/rdc/deid/config/account/account.json')
Orchestrator(client=client,vocabulary_id='PPI',concept_class_id=['Question','PPI Modifier'],dataset='raw',table='observation',fields=['value_as_number'])
import sys
del sys.modules['deid']

In [ ]:
null = None
top =   {
    "observation_id": "1040962298",
    "person_id": "562270",
    "observation_concept_id": "0",
    "observation_type_concept_id": "45905771",
    "value_as_string": "hzngxomcmeawcekthvdu",
    "value_as_concept_id": "0",
    "qualifier_concept_id": "0",
    "unit_concept_id": "0",
    "provider_id": None,
    "visit_occurrence_id": None,
    "observation_source_value": "SecondaryContactInfo_PersonOneEmail",
    "observation_source_concept_id": "1585915",
    "unit_source_value": None,
    "qualifier_source_value": None,
    "value_source_concept_id": "0",
    "value_source_value": None,
    "questionnaire_response_id": "1916632951",
    "observation_date": "-13515",
    "observation_datetime": "-13515"
  }
bot = {
    "value_as_string": "-18688",
    "person_id": "562270",
    "observation_date": "-31731",
    "observation_datetime": "-31731",
    "provider_id": null,
    "observation_concept_id": "0",
    "observation_id": "1040962363",
    "unit_source_value": null,
    "unit_concept_id": "0",
    "value_source_value": null,
    "qualifier_concept_id": "0",
    "observation_source_value": "PIIBirthInformation_BirthDate",
    "observation_type_concept_id": "45905771",
    "observation_source_concept_id": "1585259",
    "qualifier_source_value": null,
    "value_as_concept_id": "0",
    "value_source_concept_id": "0",
    "questionnaire_response_id": "1164848414",
    "visit_occurrence_id": null
  }
orig =  {
    "observation_id": "67221101",
    "person_id": "780400",
    "observation_concept_id": "4117957",
    "observation_date": "2015-09-24",
    "observation_datetime": "2015-09-24 00:00:00.000 UTC",
    "observation_type_concept_id": "8201211117",
    "value_as_number": None,
    "value_as_string": None,
    "value_as_concept_id": "0",
    "qualifier_concept_id": "0",
    "unit_concept_id": "0",
    "provider_id": "0",
    "visit_occurrence_id": "193011596",
    "observation_source_value": "Y95",
    "observation_source_concept_id": "35225031",
    "unit_source_value": None,
    "qualifier_source_value": None,
    "value_source_concept_id": None,
    "value_source_value": None,
    "questionnaire_response_id": None
  }
# [len(set(orig) - set(top)),len(set(top)-set(bot)), len(set(orig)- set(bot.keys()))]
set(orig)-set(bot)


In [ ]:
#
# This section performs suppression of PPI on a meta-table
# @TODO INsure the way in which the PPI are identified is correct. You should make sure the dates aren't suppressed.
# Dates will have to be shifted ()
#
# handler = SuppressPPI(client=client)
# r = handler.can_do('raw')
# handler.do(i_dataset='raw',table_name='observation', o_dataset='out')

In [ ]:
#
# This section performs a suppression of fields from a non-meta table
# A configureation should be provided to the class
#
# config = {"raw":{"person":['year_of_birth','month_of_birth','day_of_birth']}}
# h = SuppressEHR(client=client,config = config)
# h.can_do('raw','person')
# h.do(i_dataset='raw',o_dataset='out',table_name='person')

In [ ]:
#
# This section will perform a date shifting on PPI and EHR
# The date shifting operation will (to a certain degree) include suppression
#
h = Shift(client=client,vocabulary_id='PPI',concept_class_id=['Question','PPI Modifier'])
[h.vocabulary_id,h.concept_sql]
r = h.can_do('raw','observation')
h.get('raw','observation')['fields']
# h.do(i_dataset='raw',table_name='observation',o_dataset='out')
# h.sql

In [ ]:
# ref = client.dataset('raw').table('person')
# info = client.get_table(ref).schema
# [field.field_type for field in info]
h = DropFields(client=client)
h.can_do('raw','observation')

# str(h.policies['raw.observation']).replace('\n','')
# h.get('raw','person')

# fields = h.policies['raw.concept']
# fields
# str(fields.replace('\n','').replace('  ',' '))

In [ ]:
Orchestrator(client=client,vocabulary_id='PPI',concept_class_id=['Question','PPI Modifier'],dataset='raw',table='observation',fields=['person_id','value_as_number'])

In [ ]:
class P:
    def name(self) :
        return self.__class__.__name__.lower()
p = P()
p.name()